# FloPy Model Grids

## Notebook Setup

In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, LineString
import flopy
from flopy.discretization import StructuredGrid, VertexGrid
from flopy.utils.triangle import Triangle
from flopy.utils.voronoi import VoronoiGrid
from flopy.utils.gridgen import Gridgen
import flopy.plot.styles as styles

In [ ]:
temp_path = "./temp"
if not os.path.isdir(temp_path):
    os.mkdir(temp_path)

In [ ]:
# import all plot style information from defaults.py
from defaults import *

# Basin Example

In [ ]:
boundary_polygon = string2geom(boundary)
print("len boundary", len(boundary_polygon))
bp = np.array(boundary_polygon)

sgs = [
    string2geom(sg) for sg in (streamseg1, streamseg2, streamseg3, streamseg4)
]

fig = plt.figure(figsize=(15, 10))
ax = fig.add_subplot()
ax.set_aspect("equal")

ax.plot(bp[:, 0], bp[:, 1], "ko-")
for sg, fmt in zip(sgs, ("bo-", "ro-", "go-", "yo-")):
    print("Len segment: ", len(sg))
    sa = np.array(sg)
    ax.plot(sa[:, 0], sa[:, 1], fmt)

In [ ]:
# Create a regular MODFLOW grid
Lx = 180000
Ly = 100000
dx = dy = 2000
nrow = int(Ly / dy)
ncol = int(Lx / dx)
print(Lx, Ly, nrow, ncol)
delr = np.array(ncol * [dx])
delc = np.array(nrow * [dy])
regular_grid = StructuredGrid(delr=delr, delc=delc, xoff=0.0, yoff=0.0)

figsize = (17.15 / 2.54, (Ly / Lx) * 17.15 / 2.54)
fig = plt.figure(figsize=figsize)
ax = fig.add_subplot()
ax.set_aspect("equal")
regular_grid.plot(ax=ax)
ax.plot(bp[:, 0], bp[:, 1], "k-")
for sg in sgs:
    sa = np.array(sg)
    ax.plot(sa[:, 0], sa[:, 1], "b-")

In [ ]:
def set_idomain(grid, boundary):
    from flopy.utils.gridintersect import GridIntersect
    from shapely.geometry import Polygon

    ix = GridIntersect(grid, method="vertex", rtree=True)
    result = ix.intersect(Polygon(boundary))
    idx = [coords for coords in result.cellids]
    idx = np.array(idx, dtype=int)
    nr = idx.shape[0]
    if idx.ndim == 1:
        idx = idx.reshape((nr, 1))
    print(idx.shape, idx.ndim)
    idx = tuple([idx[:, i] for i in range(idx.shape[1])])
    # idx = (idx[:, 0], idx[:, 1])
    idomain = np.zeros(grid.shape[1:], dtype=int)
    idomain[idx] = 1
    idomain = idomain.reshape(grid.shape)
    grid.idomain = idomain

In [ ]:
# Create a regular MODFLOW grid
Lx = 180000
Ly = 100000
dx = dy = 1000
nrow = int(Ly / dy)
ncol = int(Lx / dx)
print(Lx, Ly, nrow, ncol)
delr = np.array(ncol * [dx])
delc = np.array(nrow * [dy])
regular_grid = StructuredGrid(nlay=1, delr=delr, delc=delc, xoff=0.0, yoff=0.0)

set_idomain(regular_grid, boundary_polygon)

fig = plt.figure(figsize=figsize)
ax = fig.add_subplot()
pmv = flopy.plot.PlotMapView(modelgrid=regular_grid)
ax.set_aspect("equal")
pmv.plot_grid()
pmv.plot_inactive()
# regular_grid.plot(ax=ax, )
ax.plot(bp[:, 0], bp[:, 1], "k-")
for sg in sgs:
    sa = np.array(sg)
    ax.plot(sa[:, 0], sa[:, 1], "b-")

In [ ]:
ixs = flopy.utils.GridIntersect(regular_grid, method="structured")

In [ ]:
chd_intersection = []
chd_cellids = []
for sg in sgs:
    v = ixs.intersect(LineString(sg), sort_by_cellid=True)
    chd_intersection.append(v)
    chd_cellids += v["cellids"].tolist()

In [ ]:
regular_grid.nrow, regular_grid.ncol

In [ ]:
regular_grid.delc

In [ ]:
chd_elev = []
for i, j in chd_cellids:
    v = (0, i, j, (100.0 / regular_grid.ncol) * (regular_grid.ncol - j + 1))
    if v not in chd_elev:
        chd_elev.append(v)

In [ ]:
sim = flopy.mf6.MFSimulation(
    sim_name="create_topo", sim_ws="temp_topo", exe_name="mf6"
)

tdis = flopy.mf6.ModflowTdis(sim)
ims = flopy.mf6.ModflowIms(sim, linear_acceleration="bicgstab")
gwf = flopy.mf6.ModflowGwf(sim, newtonoptions="NEWTON UNDER_RELAXATION")

dis = flopy.mf6.ModflowGwfdis(
    gwf,
    nlay=1,
    nrow=regular_grid.nrow,
    ncol=regular_grid.ncol,
    delr=regular_grid.delr,
    delc=regular_grid.delc,
    idomain=regular_grid.idomain,
    top=100,
    botm=-100,
)

ic = flopy.mf6.ModflowGwfic(gwf, strt=50.0)
npf = flopy.mf6.ModflowGwfnpf(gwf, icelltype=1)
rch = flopy.mf6.ModflowGwfrcha(gwf, recharge=0.00001)
chd = flopy.mf6.ModflowGwfchd(gwf, stress_period_data=chd_elev)
oc = flopy.mf6.ModflowGwfoc(
    gwf,
    head_filerecord=f"{gwf.name}.hds",
    saverecord=[("HEAD", "ALL")],
)

In [ ]:
sim.write_simulation()
sim.run_simulation()

In [ ]:
head = gwf.output.head().get_data()

In [ ]:
v = head.copy()
v[v == 1e30] = 0.0
v.max()

In [ ]:
figsize = (8.25 / 2.54, (Ly / Lx) * 8.25 / 2.54)
levels = np.arange(0, 110, 10)
contour_dict = {"levels": levels, "linewidths": 1.0, "colors": "white"}
clabel_dict = {
    "inline": True,
    "fmt": "%1.0f",
    "fontsize": 6,
    "inline_spacing": 0.5,
}
font_dict = {"fontsize": 5, "color": "white"}

with styles.USGSMap():
    fig = plt.figure(figsize=figsize, constrained_layout=True)

    mv = flopy.plot.PlotMapView(model=gwf)
    v = mv.plot_array(head, masked_values=(1.0e30,))
    mv.plot_grid(lw=0.25, color="0.5")
    cg = mv.contour_array(head, masked_values=(1.0e30,), **contour_dict)
    mv.ax.clabel(cg, cg.levels, **clabel_dict)
    mv.plot_inactive(color_noflow="gray")
    mv.ax.plot(bp[:, 0], bp[:, 1], "k-", lw=2.0)
    for sg in sgs:
        sa = np.array(sg)
        mv.ax.plot(sa[:, 0], sa[:, 1], "b-", lw=1.0)
    mv.ax.set_xticks(np.arange(0, 200000, 50000))
    mv.ax.set_xticklabels(np.arange(0, 200, 50))
    mv.ax.set_yticks(np.arange(0, 150000, 50000))
    mv.ax.set_yticklabels(np.arange(0, 150, 50))
    mv.ax.set_xlabel("x position (km)")
    mv.ax.set_ylabel("y position (km)")

    cax = fig.add_axes([0.70, 0.85, 0.2, 0.025])
    cbar = plt.colorbar(
        v,
        orientation="horizontal",
        shrink=0.5,
        cax=cax,
        ticks=[25, 50, 75, 100],
    )
    cbar.ax.tick_params(
        labelsize=5, labelcolor="white", color="white", length=4.5, pad=2
    )
    cbar.ax.set_title("elevation (m)", pad=2.5, loc="left", fontdict=font_dict)

    fpth = os.path.join("..", "doc", "figures", "fine_topo.png")
    plt.savefig(fpth, dpi=300);

In [ ]:
# np.savetxt("grid_data/fine_topo.asc", head[0], fmt="%.8g")